In [1]:
import utils

utils.load_env()

from langchain.llms import OpenAI

# Create an instance of the OpenAI class
openai = OpenAI(model_name="text-davinci-003")

# Get the maximum context size for the model
max_context_size = openai.modelname_to_contextsize(openai.model_name)

# Set the max_tokens attribute to a value within the model's maximum context length
openai.max_tokens = max_context_size - 1  # or any value less than max_context_size

# Alternatively, you can use the max_tokens_for_prompt method
prompt = "Tell me a joke."*800
openai.max_tokens = openai.max_tokens_for_prompt(prompt)
openai.max_tokens

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


97

In [4]:
from copy import copy
tools_outputs="asdfdfaf"

def get_tools_output():
    global tools_outputs
    a = copy(tools_outputs)
    tools_outputs = ""
    return a
    
get_tools_output()

'asdfdfaf'